In [1]:
pip install pyevmasm

In [2]:
import pandas as pd
import numpy as np
import requests
import certifi
from pyevmasm import instruction_tables, disassemble_hex, disassemble_all, assemble_hex
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [10]:
addresses = pd.read_csv(r"/content/Contract Adresses.csv")
#left_ds = pd.read_csv(r"/Users/lucapennella/Desktop/Smart Ponzi/Dataset/dataset.csv")

In [7]:
addr_list = []
bytecode_list = {}
opcode_list = {}

In [21]:
addresses.shape

(7962, 1)

In [11]:
addresses.head()

,address
0,0x0006157838d5a6b33ab66588a6a693a57c869999
1,0x001a589dda0d6be37632925eaf1256986b2c6ad0
2,0x00674045bb7c17f0aa1cde34780d6c51af548728
3,0x006bea43baa3f7a6f765f14f10a1a1b08334ef45
4,0x00a9f7d093c46d95f0318e4a6ffc6ed68f73044c


In [23]:
addresses['address'].nunique()

7962

In [12]:
for addr in addresses['address']: #creo una lista con tutti gli address
      addr_list.append(addr)

In [13]:
for addr in addr_list: # ciclo su tutti gli indirizzi presenti nella lista
    normal_tx_url = "https://api.etherscan.io/api?module=account&action=txlist&address=" + addr + \
                   "&startblock=0&endblock=99999999&page=1&offset=10000&sort=asc&apikey=SAKUKN5G8E7R2IYIWI2IVS7ZRKXDAX8NE6"
    response_normal = requests.get(normal_tx_url, verify = certifi.where()) # la get restituisce un Response obj
    address_content_normal = response_normal.json() # la risposta e' in formato json, si ottiene un dizionario python
    result_normal = address_content_normal.get('result') # facendo la get e si ottiene tutto cio' che sta dopo result, si ottiene una lista

    for t in result_normal:
        if (t['to'] == ''): # prendo l'oggetto json con la creazione del contratto che ha il to = ''
            bytecode = (t['input']) # aggiungo il valore della chiave input che è uguale al bytecode
            bytecode_list[addr]=bytecode # faccio append con l'obiettivo di creare una lista con tutti i bytecode.

In [17]:
bytecode_df = pd.DataFrame(list(bytecode_list.items()), columns=['address', 'bytecode'])

In [22]:
bytecode_df.shape

(7753, 2)

In [20]:
bytecode_df.to_csv('bytecode_list.csv', index=False)

In [26]:
risultato = pd.merge(addresses, bytecode_df, on='address', how='left')

In [28]:
risultato

,address,bytecode
0,0x0006157838d5a6b33ab66588a6a693a57c869999,0x60606040523415600e57600080fd5b5b603680601c60...
1,0x001a589dda0d6be37632925eaf1256986b2c6ad0,0x606060405260018054600160a060020a031916732658...
2,0x00674045bb7c17f0aa1cde34780d6c51af548728,0x606060405234156200001057600080fd5b6040516200...
3,0x006bea43baa3f7a6f765f14f10a1a1b08334ef45,0x60a0604052600960608190527f546f6b656e20302e31...
4,0x00a9f7d093c46d95f0318e4a6ffc6ed68f73044c,0x60a0604052601560608190527f41686f6f6c65652054...
...,...,...
7957,0xfd784da5c740c617aafb80399fa81b86e1da99a5,0x6060604052341561000c57fe5b5b5b33600360006101...
7958,0xfd7e33bc01a7493b189ddfdcc047500463be573a,0x6060604052341561000f57600080fd5b600080546001...
7959,0xfd82ffc0d08dcfd902cbf6e48550033b01f919cc,0x60606040526000600255604080519081016040526005...
7960,0xfd9683e9f2c62e08b6bf68123e18e527efa8fbbc,0x606060405260048054600160a060020a031990811690...


In [29]:
risultato['bytecode'].isna().sum()

209

In [30]:
righe_con_null  = risultato[risultato.isnull().any(axis=1)]

In [31]:
righe_con_null

,address,bytecode
49,0x046e6bea4d53727ccedc2c252ea61d042ff7aba6,NaN
103,0x0a2b7fd09f82b221f4704dc2501eb4c12cce346d,NaN
136,0x0BeA4f7Ee7DbD9CCcFBa4b2E7A178dfC1e877723,NaN
321,0x1dfa9bde6b5bf21536b903ff06d9bd0b64fdff90,NaN
360,0x22164e957ac4c0cb0f19c49b05e627675436dfe1,NaN
...,...,...
7805,0xf56669a5f6d40a3867fdc5befae6f50e02961365,NaN
7842,0xf7b098298f7c69fc14610bf71d5e02c60792894c,NaN
7843,0xf7bd2f34d3be33b09b8e0e05ae0a1773b2a56cdf,NaN
7874,0xf95730d1901c9ee02994091e28839eb1a5d3b3e2,NaN


In [33]:
righe_con_null.to_csv('address_without_byt.csv', index=False)

In [14]:
# creo una lista di bytecode
#for k in bytecode_list.keys():
#    opcode = disassemble_hex(bytecode_list[k])# conversione bytecode in opcode
#    opcode_list[k] = opcode

KeyboardInterrupt: ignored

In [ ]:
#df_full_comm = pd.DataFrame()
#for addr in opcode_list.keys():
#    row = []
#    list_comm = pd.Series(opcode_list[addr].split('\n'))
#    list_comm = list_comm.str.replace("PUSH.*", "PUSH")
#    list_comm = list_comm.str.replace("DUP.*", "DUP")
#    list_comm = list_comm.str.replace("SWAP.*", "SWAP")
#    list_comm = list_comm.str.replace("LOG.*", "LOG")
#    df_comm = pd.DataFrame(list_comm.value_counts())
#    df_comm = df_comm.T
#    df_comm['address'] = [addr]
#    df_full_comm = pd.concat([df_full_comm,df_comm] )

In [ ]:
#df_full_comm

In [ ]:
#df_full_comm.to_csv(r"/Users/lucapennella/Desktop/Smart Ponzi/bytecode_decode.csv" , index=False)

In [25]:
#df_full = pd.merge(left_ds, df_full_comm, how='left', on = 'address')
#df_full.to_csv(r"/Users/lucapennella/Desktop/Smart Ponzi/bytecode_decode.csv" , index=False)

NameError: ignored